# Практическое задание


## Задача

Есть переписка ТП с пользователем.
Переписка содержит все данные по решению инцидента
- описание и уточнение проблемы,
- обсуждение вариантов решения,
- итоговое решение

На основе переписки формировать два текста:
1. краткое описание проблемы (возможно, с выделением ключевых слов),
2. шаги решения (инструкция).

Будет использоваться механизм поиска похожих обращений (по описанию проблемы)


## Ожидаемое решение
Функция:
```chat_process(переписка | список переписок): -> solution```


## Подробный разбор
### 1. Извлечение данных из .docx файлов

### 2. Обработка входных данных:
- Создание класса переписки
    - Имя компании
    - Массив номеров обращений
    - Список пар [отправитель, текст сообщения]
- Сохранение переписок из .docx в массив

### 3. Подготовка данных к обработке ИИ
#### 3.1. Очистка и форматирование чатов
#### 3.2. Подготовка директории и загрузка обработанных текстов

#### 4. Описание основного процесса обработки переписки/ переписок:
- Получение данных в готовом виде (класс переписки/ список класса переписки)
- Обработка данных по списку или единичному экземпляру
- Запаковка в специальную структуру
- Возвращение списка

#### 4.1. Описание структуры для данных на выход:
- Имя компании
- Массив номеров обращений
- Краткое описание проблемы
- Ключевые слов
- Решение проблемы

#### 4.2. Описание основной функции:
- Подготовка места для загрузки текста для обработки
- Создание конфигурационной области:
    - API ИИ
    - Аббревиатуры для понимания ИИ
    - Инструкции для обработки чатов
- Запросы инструктивным промптом к ИИ по АПИ для:
    - формирования краткого описания проблемы (по шаблону?)
    - нахождения ключевых слов (с примерами)
    - нахождения шагов решения


### 1. Функция извлечения .docx из папки

In [272]:
%pip install python-docx

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [273]:
from os import listdir
from docx import Document


def load_docx_from_project(path_in_project='input_data/'):
    documents = []

    for filename in listdir(path_in_project):
        if filename.endswith('.docx'):
            full_path = path.join(path_in_project, filename)
            try:
                document = Document(full_path)
                documents.append([filename, document])
            except Exception as e:
                print(f"Не удалось загрузить {filename}: {e}")

    return documents


# Извлечение файлов .docx
docs = load_docx_from_project()


### 2. Обработка входных данных

#### 2.1. Описание класса Chat

In [274]:
from dataclasses import dataclass


@dataclass
class Message:
    sender: str
    text: str


@dataclass
class Chat:
    messages: list[Message]
    name: str = None
    numbers: list[str] = None


#### 2.2. Изменение типа данных (из .docx в Chat)

In [275]:
def convert_from_docx(document: tuple[str, Document]):
    name = document[0][:-5]

    table = document[1].tables[0]
    numbers: list[str] = []
    messages: list[Message] = []
    for row in table.rows[1:]:
        cells = row.cells
        message = Message(cells[1].text, cells[2].text)

        numbers.append(cells[0].text)
        messages.append(message)
    numbers = set(numbers)

    chat = Chat(
        name=name,
        numbers=numbers,
        messages=messages
    )
    '''
    print(f"|{'Компания':<16}|", chat.name,
          f"\n|{'№ обращений':<16}|", chat.numbers,
          f"\n|{'Отправитель':<16}|", chat.messages[0].sender,
          f"\n|{'Текст сообщения':<16}|\n", chat.messages[0].text)
    '''
    return chat


# Изменение типа переменных
chats: list[Chat] = []
for doc in docs:
    chat = convert_from_docx(doc)
    chats.append(chat)


### 3. Подготовка данных к обработке ИИ

#### 3.1. Очистка и форматирование чатов

In [276]:
# Функция очистки текста
REMOVE_CHARS = {
    0x00A0: None,  # NBSP – неразрывный пробел
    0x00AD: None,  # SHY – мягкий перенос
    0x200B: None,  # ZWSP – нулевой пробел
    0xFEFF: None   # BOM – метка порядка байтов
}


from re import sub


def clear_text(
    text: str,
    symbols_to_remove: dict[int, None],
    to_lower: bool = False,
    normalize_spaces: bool = True
) -> str:
    text = text.translate(symbols_to_remove)
    text = sub(r'\n+', '\n', text)
    text = sub(r'[ \t]+\n', '\n', text)

    if normalize_spaces:
        text = sub(r'[ \t]{2,}', ' ', text)

    text = "\n".join(line.strip() for line in text.splitlines())

    if to_lower:
        text = text.lower()

    return text.strip()


def format_chat(chat: Chat) -> tuple[str, str]:
    dialog = ""

    for message in chat.messages:
        cleared_text = clear_text(message.text, REMOVE_CHARS)
        dialog += (message.sender + ': ' + '\n' +
                   cleared_text + '\n\n')

    name_data = tuple([chat.name, dialog])
    return name_data


texts_to_process: list[tuple[str, str]] = []
for chat in chats:
    texts_to_process.append(format_chat(chat))


#### 3.2. Подготовка директории и загрузка обработанных текстов

In [277]:
from os import makedirs


def ensure_folder_exists(folder_path: str = "to_process") -> None:
    makedirs(folder_path, exist_ok=True)


def write_text_to_file(
    name_data: tuple[str, str],
    folder_path: str = "to_process"
) -> None:
    file_path = folder_path + '/' + name_data[0] + ".txt"
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(name_data[1])


ensure_folder_exists()
for data in texts_to_process:
    write_text_to_file(data)


### 4. Описание структуры для данных на выход:

In [278]:
@dataclass
class ProblemWithSolution:
        description: str
        keywords: list[str]
        solution: str
        name: str = "untitled"
        numbers: list[str] = None


### 4.2. Подробное описание фрагмента обработки из основной функции:
- Запросы инструктивным промптом к ИИ по АПИ для:
    - формирования краткого описания проблемы (по шаблону?)
    - нахождения ключевых слов (с примерами)
    - нахождения шагов решения
- Одновременно все 3 запроса (можно без обхода GIL, тк IO)

In [279]:
def load_data(instruction_path: str) -> str:
    with open(instruction_path, "r", encoding="utf-8") as f:
        return f.read()

In [280]:
from json import load
from dataclasses import dataclass
from typing import Optional


@dataclass
class InstructionBlock:
    role: str
    instruction: str
    context: str
    format: Optional[str]
    max_tokens: int


@dataclass
class LLMRequest:
    instruction_block: InstructionBlock
    task: str

    def to_prompt(self) -> list[dict[str, str]]:
        prompt = (
            f"{self.instruction_block.instruction}\n\n"
            f"Контекст:\n{self.instruction_block.context}\n\n"
            f"Формат вывода:\n{self.instruction_block.format}\n\n"
            f"Ограничение по токенам:\n{self.instruction_block.max_tokens}\n\n"
            f"Входные данные:\n{self.task}"
        )
        return [
            {"role": "system", "content": self.instruction_block.role},
            {"role": "user", "content": prompt}
        ]

    @property
    def max_tokens(self) -> int:
        return self.instruction_block.max_tokens


def load_instruction_file(json_path: str):
    with open(json_path, "r", encoding="utf-8") as f:
        data = load(f)

    abbreviations = "\n".join(data["context"])
    role = data["role"]

    def make_instruction(key: str) -> InstructionBlock:
        item = data["instructions"][key]
        return InstructionBlock(
            role=role,
            instruction=item["instruction"],
            context=abbreviations,
            format=item["response_format"],
            max_tokens=item["max_tokens"]
        )

    return {
        "abbreviations": abbreviations,
        "instruction_for_description": make_instruction("description"),
        "instruction_for_keywords": make_instruction("keywords"),
        "instruction_for_solution": make_instruction("solution"),
    }

In [281]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [282]:
from openai import OpenAI


class AIModelAPI:
    def __init__(self, api: str, url: str, model_name: str):
        self.api = api
        self.url = url
        self.model_name = model_name
        self.client = OpenAI(api_key=self.api, base_url=self.url)

    def get_response(self, request: LLMRequest,
                     max_tokens: int = 50, temperature: float = 0.1):
        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=request.to_prompt(),
            stream=False,
            max_tokens=max_tokens,
            temperature=temperature
        )
        return response.choices[0].message.content


In [283]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from os import path

  
def chat_process(
        model: AIModelAPI,
        path_to_instruct_json: str,
        path_to_task_txt: str,
        multi_thread: bool = False
) -> ProblemWithSolution:

    def _call_request(request: LLMRequest) -> str:
        result = model.get_response(request, max_tokens=request.max_tokens)
        return result

    def _check_json_and_txt(path_json: str, path_to_txt: str,) -> None:
        shaped_path_task = path.splitext(path.basename(path_to_txt))
        format_of_file_task = shaped_path_task[1]
        if format_of_file_task != ".txt":
            raise ValueError(f"Ожидаемый формат файла задачи - .txt, получен - {format_of_file_task}")
        
        shaped_path_instruct = path.splitext(path.basename(path_json))
        format_of_file_instruct = shaped_path_instruct[1]
        if format_of_file_instruct != ".json":
            raise ValueError(f"Ожидаемый формат файла инструкций - .json, получен - {format_of_file_task}")
    
    
    _check_json_and_txt(path_json=path_to_instruct_json, path_to_txt=path_to_task_txt)
    
    name = path.splitext(path.basename(path_to_task_txt))[0]
    
    general_task = load_data(path_to_task_txt)

    instructions = load_instruction_file(path_to_instruct_json)
    i_description = instructions["instruction_for_description"]
    i_keywords    = instructions["instruction_for_keywords"]
    i_solution    = instructions["instruction_for_solution"]

    r_description = LLMRequest(i_description, task=general_task)
    r_keywords = LLMRequest(i_keywords,       task=general_task)
    r_solution = LLMRequest(i_solution,       task=general_task)

    tasks = {
        "description": r_description,
        "keywords":    r_keywords,
        "solution":    r_solution
    }

    results: dict[str, str] = {}

    if multi_thread:
        with ThreadPoolExecutor(max_workers=3) as executor:
            future_to_label = {
                executor.submit(_call_request, request): label
                for label, request in tasks.items()
            }
            for future in as_completed(future_to_label):
                label = future_to_label[future]
                try:
                    results[label] = future.result(timeout=30)
                except Exception as e:
                    results[label] = f"Ошибка: {e}"

    else:
        for label, request in tasks.items():
            try:
                results[label] = _call_request(request)
            except Exception as e:
                results[label] = f"Ошибка: {e}"


    for key in tasks: results.setdefault(key, "")

    return ProblemWithSolution(
        name=name,
        description=results["description"],
        keywords=results["keywords"].split(),
        solution=results["solution"]
    )


In [284]:
from os import getenv
from dotenv import load_dotenv


load_dotenv()

path_to_task_txt = "to_process/ПАКС ООО.txt"
path_to_instruct_json = "prompt_data.json"

model_url = "https://api.deepseek.com"
model_name = "deepseek-chat"
API_AI = getenv("API_DS")

# model_url = "https://api.openai.com/v1"
# model_name = "gpt-4-turbo"
# API_AI = getenv("API_GPT")

model = AIModelAPI(API_OPENAI, model_url, model_name)


In [285]:
sol = chat_process(
    model=model,
    path_to_instruct_json=path_to_instruct_json,
    path_to_task_txt=path_to_task_txt,
    multi_thread=False
)


In [286]:
separator = "-" * 50

print(f"{sol.name}              \n{separator}\n"
      f"{sol.description}       \n{separator}\n"
      f"{' '.join(sol.keywords)}\n{separator}\n"
      f"{sol.solution}"                         )

ПАКС ООО              
--------------------------------------------------
Лицензия Скан-Архива допускает использование модуля для ERP после перехода с УПП, требуется интеграция подсистемы.       
--------------------------------------------------
ERP УПП Скан-Архив СА лицензия рабочие места конфигурация подсистема расширение интеграция дистрибутивы инструкции настройки установка библиотеки Microsoft VCRedist шрифты штрихкод терминальный сервер лицензии файл ключ активация р
--------------------------------------------------
1. Скачайте и установите библиотеки Microsoft VCRedist 2012, 2013 и 2015-2019 (Клиент):
   1.1. vcredist 2012 (x86, x64): [ссылка](https://www.microsoft.com/ru-ru/download/details.aspx?id=30679)
   1.2. vcredist 2013 (x86, x64): [ссылка](https://www.microsoft.com/ru-RU/download/details.aspx?id=40784)
   1.3. vcredist 2017 (x86, x64): [ссылка](https://support.microsoft.com/ru-ru/help/2977003/the-latest-supported-visual-c-downloads)

2. Если работаете в терминальном р